# Explicabilidad XAI — Scripts 7, 8 y 9

**Universidad de Especialidades Espiritu Santo (UEES)**  
**Maestria en Inteligencia Artificial — Aprendizaje Automatico**

**Estudiantes:**
- Ing. Gonzalo Mejia Alcivar
- Ing. Jorge Ortiz Merchan
- Ing. David Perugachi Rojas

---

Este notebook integra los tres scripts de explicabilidad XAI del proyecto:

| Script | Tecnicas | Imagenes |
|--------|----------|----------|
| `7_ExplicabilidadSHAP.py` | SHAP + Permutation Feature Importance | 32–39 |
| `8_ExplicabilidadPDP_Arbol.py` | PDP + ICE + Arbol de Decision | 40–47 |
| `9_VisualizacionesXAI_Integradas.py` | Panel integrado: impacto, comparacion, casos | 48–56 |

> **Prerequisito:** ejecutar los scripts 1–6 para generar los modelos `.pkl` en `Models/`.

---
## Script 7 — SHAP y Permutation Feature Importance

**Tecnicas aplicadas:**
- **SHAP (SHapley Additive exPlanations):** `TreeExplainer` sobre Random Forest. Calcula la contribucion de cada feature a cada prediccion usando teoria de juegos cooperativos.
- **Permutation Feature Importance (PFI):** mide la caida en F1-Score al permutar aleatoriamente cada feature (5 repeticiones). Aplicado a los 3 modelos.

**Resultados clave:**
- Feature mas importante (SHAP global): `UtilidadNeta`
- Los 3 modelos coinciden en el mismo top-3 con PFI
- Imagenes generadas: `32_shap_barplot_global.png` a `39_shap_vs_pfi_comparativa.png`

In [ ]:
# -*- coding: utf-8 -*-
import os
import warnings

import matplotlib
matplotlib.use("Agg")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import joblib
import shap

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.inspection import permutation_importance

warnings.filterwarnings("ignore")
plt.rcParams.update({"figure.max_open_warning": 0})
print("Librerias importadas correctamente.")

In [ ]:
# =========================================================
# 1. CONFIGURACION DE RUTAS
# =========================================================
BASE_DIR = os.path.dirname(os.path.abspath("."))
# Si ejecutas desde la raiz del proyecto, usar directamente:
BASE_DIR = os.path.abspath(".")
DATA_PATH = os.path.join(BASE_DIR, "Data", "DataSet2024.csv")
MODELS_DIR = os.path.join(BASE_DIR, "Models")
RESULTS_DIR = os.path.join(BASE_DIR, "results")
os.makedirs(RESULTS_DIR, exist_ok=True)

def save_fig(fig, name):
    path = os.path.join(RESULTS_DIR, name)
    fig.savefig(path, dpi=150, bbox_inches="tight", facecolor="white")
    plt.close(fig)
    print(f"  -> Guardado: {path}")

print(f"BASE_DIR : {BASE_DIR}")
print(f"DATA_PATH: {DATA_PATH}")
print(f"MODELS   : {MODELS_DIR}")
print(f"RESULTS  : {RESULTS_DIR}")

In [ ]:
# =========================================================
# 2. CARGA DE MODELOS Y DATOS
# =========================================================
print("=" * 60)
print("  EXPLICABILIDAD XAI - SHAP Y PERMUTATION IMPORTANCE")
print("  Dataset: Empresas del Ecuador - 2024")
print("=" * 60)

print("\n  Cargando modelos exportados...")
dt_model  = joblib.load(os.path.join(MODELS_DIR, "arbol_decision.pkl"))
rf_model  = joblib.load(os.path.join(MODELS_DIR, "random_forest.pkl"))
svm_model = joblib.load(os.path.join(MODELS_DIR, "svm.pkl"))
scaler    = joblib.load(os.path.join(MODELS_DIR, "scaler.pkl"))
le_target = joblib.load(os.path.join(MODELS_DIR, "label_encoder_target.pkl"))
le_sector = joblib.load(os.path.join(MODELS_DIR, "label_encoder_sector.pkl"))
feature_cols = joblib.load(os.path.join(MODELS_DIR, "feature_columns.pkl"))

clases = list(le_target.classes_)
print(f"  Modelos cargados: Arbol de Decision, SVM, Random Forest")
print(f"  Features: {feature_cols}")
print(f"  Clases:   {clases}")

In [ ]:
# =========================================================
# 3. RECONSTRUCCION DEL CONJUNTO DE PRUEBA
# =========================================================
print("\n  Reconstruyendo conjunto de prueba...")

df = pd.read_csv(DATA_PATH, sep=";", encoding="utf-8-sig", engine="python", on_bad_lines="skip")
df.columns = (
    df.columns.str.strip()
    .str.replace("\n", "_", regex=False)
    .str.replace(" ", "_", regex=False)
    .str.replace(".", "", regex=False)
)

col_ano = [c for c in df.columns if df[c].nunique() == 1 and df[c].dtype in ["int64", "float64"]]
if col_ano:
    df = df.drop(columns=col_ano)

cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
for col in df.columns:
    if col not in cat_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

epsilon = 1e-7
df["Margen_Neto"] = df["UtilidadNeta"] / (df["IngresosTotales"] + epsilon)
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["Margen_Neto"])
df["Desempeno"] = pd.qcut(df["Margen_Neto"], q=3, labels=["Bajo", "Medio", "Alto"], duplicates="drop")

df["Sector"] = le_sector.transform(df["Sector"].astype(str))
df["Desempeno_cod"] = le_target.fit_transform(df["Desempeno"])

X = df[feature_cols]
y = df["Desempeno_cod"]
X_scaled = pd.DataFrame(scaler.transform(X), columns=feature_cols, index=X.index)

_, X_test, _, y_test = train_test_split(
    X_scaled, y, test_size=0.20, random_state=42, stratify=y
)

SHAP_SAMPLE = min(2000, len(X_test))
X_shap = X_test.sample(n=SHAP_SAMPLE, random_state=42)

print(f"  Conjunto de prueba: {X_test.shape[0]:,} registros")
print(f"  Muestra para SHAP:  {SHAP_SAMPLE:,} registros")

In [ ]:
# =========================================================
# 4. TECNICA 1 — SHAP (SHapley Additive exPlanations)
# =========================================================
print("\n  Calculando SHAP values para Random Forest...")

explainer_rf = shap.TreeExplainer(rf_model)
shap_values_raw = explainer_rf.shap_values(X_shap)

# Normalizar estructura: SHAP >= 0.42 retorna array 3D (n_muestras, n_features, n_clases)
if isinstance(shap_values_raw, np.ndarray) and shap_values_raw.ndim == 3:
    n_clases_shap = shap_values_raw.shape[2]
    shap_values_rf = [shap_values_raw[:, :, i] for i in range(n_clases_shap)]
elif isinstance(shap_values_raw, list):
    shap_values_rf = shap_values_raw
else:
    shap_values_rf = [shap_values_raw]

print(f"  Shape: {len(shap_values_rf)} clases x {shap_values_rf[0].shape[0]} muestras x {shap_values_rf[0].shape[1]} features")

sns.set_style("whitegrid")
palette_clase = {"Alto": "#27ae60", "Bajo": "#e74c3c", "Medio": "#f39c12"}

In [ ]:
# --- 4.1 SHAP Bar Plot: importancia media absoluta global ---
print("[1/5] SHAP Bar Plot - Importancia media absoluta global...")

mean_abs_shap = np.mean([np.abs(shap_values_rf[i]).mean(axis=0) for i in range(len(clases))], axis=0)
shap_importance = pd.DataFrame({
    "Feature": feature_cols,
    "SHAP_mean_abs": mean_abs_shap
}).sort_values("SHAP_mean_abs", ascending=True)

fig, ax = plt.subplots(figsize=(10, 7))
colors_bar = plt.cm.RdYlGn(shap_importance["SHAP_mean_abs"] / shap_importance["SHAP_mean_abs"].max())
bars = ax.barh(shap_importance["Feature"], shap_importance["SHAP_mean_abs"],
               color=colors_bar, edgecolor="black", linewidth=0.5)
for bar, val in zip(bars, shap_importance["SHAP_mean_abs"]):
    ax.text(val + shap_importance["SHAP_mean_abs"].max() * 0.01,
            bar.get_y() + bar.get_height() / 2,
            f"{val:.4f}", va="center", fontsize=9)
ax.set_xlabel("Media del valor SHAP absoluto (impacto en la prediccion)", fontsize=11)
ax.set_title("SHAP - Importancia Global de Features\n(Random Forest | Promedio todas las clases)",
             fontsize=13, fontweight="bold")
ax.grid(axis="x", linestyle="--", alpha=0.5)
fig.tight_layout()
save_fig(fig, "32_shap_barplot_global.png")

In [ ]:
# --- 4.2 SHAP Beeswarm Plot por clase ---
print("[2/5] SHAP Beeswarm Plot por clase...")

fig, axes = plt.subplots(1, 3, figsize=(22, 7))

for i, (clase, ax) in enumerate(zip(clases, axes)):
    mean_abs_clase = np.abs(shap_values_rf[i]).mean(axis=0)
    order_idx = np.argsort(mean_abs_clase)[::-1]
    top_n = min(8, len(feature_cols))
    top_idx = order_idx[:top_n]

    shap_vals_clase = shap_values_rf[i][:, top_idx]
    feat_vals_clase = X_shap.iloc[:, top_idx]
    feat_names_top  = [feature_cols[j] for j in top_idx]

    for k, feat_name in enumerate(feat_names_top):
        y_pos   = top_n - 1 - k
        sv      = shap_vals_clase[:, k]
        fv      = feat_vals_clase.iloc[:, k].values
        fv_norm = (fv - fv.min()) / (fv.max() - fv.min() + 1e-9)
        colors  = plt.cm.coolwarm(fv_norm)
        jitter  = np.random.RandomState(42).uniform(-0.2, 0.2, len(sv))
        ax.scatter(sv, y_pos + jitter, c=colors, alpha=0.4, s=8, linewidths=0)

    ax.axvline(x=0, color="black", linewidth=0.8, linestyle="--")
    ax.set_yticks(range(top_n))
    ax.set_yticklabels(feat_names_top[::-1], fontsize=9)
    ax.set_xlabel("Valor SHAP", fontsize=10)
    ax.set_title(f"Clase: {clase}", fontsize=12, fontweight="bold",
                 color=palette_clase[clase])
    ax.grid(axis="x", linestyle="--", alpha=0.3)

fig.suptitle("SHAP Beeswarm - Impacto de Features por Clase\n"
             "(Azul = valor bajo, Rojo = valor alto de la feature)",
             fontsize=14, fontweight="bold", y=1.02)
sm   = plt.cm.ScalarMappable(cmap="coolwarm", norm=plt.Normalize(0, 1))
sm.set_array([])
cbar = fig.colorbar(sm, ax=axes, orientation="vertical", fraction=0.02, pad=0.04)
cbar.set_label("Valor de la feature (normalizado)", fontsize=10)
fig.tight_layout()
save_fig(fig, "33_shap_beeswarm_por_clase.png")

In [ ]:
# --- 4.3 SHAP Waterfall Plot individual ---
print("[3/5] SHAP Waterfall Plot - Explicacion individual...")

fig, axes = plt.subplots(1, 3, figsize=(22, 7))
y_shap_pred = rf_model.predict(X_shap)

for i, (clase_idx, clase_nombre) in enumerate(enumerate(clases)):
    ax   = axes[i]
    mask = y_shap_pred == clase_idx
    if mask.sum() == 0:
        ax.text(0.5, 0.5, f"Sin muestras\npredichas como\n{clase_nombre}",
                ha="center", va="center", transform=ax.transAxes, fontsize=12)
        ax.set_title(f"Clase: {clase_nombre}", fontsize=12)
        continue

    sample_idx = np.where(mask)[0][0]
    sv         = shap_values_rf[clase_idx][sample_idx]
    ev         = explainer_rf.expected_value
    base_val   = ev[clase_idx] if hasattr(ev, "__len__") else float(ev)

    order      = np.argsort(np.abs(sv))[::-1]
    top_k      = min(8, len(sv))
    sv_top     = sv[order[:top_k]]
    fn_top     = [feature_cols[j] for j in order[:top_k]]
    cumsum     = np.cumsum(sv_top)
    starts     = np.concatenate([[base_val], base_val + cumsum[:-1]])
    colors_wf  = ["#e74c3c" if v > 0 else "#3498db" for v in sv_top]

    ax.barh(range(top_k), sv_top, left=starts, color=colors_wf,
            edgecolor="black", linewidth=0.5, height=0.6)
    ax.axvline(x=base_val, color="gray", linewidth=1, linestyle="--",
               alpha=0.7, label=f"Base: {base_val:.3f}")
    ax.axvline(x=base_val + cumsum[-1], color="black", linewidth=1.5,
               label=f"Pred: {base_val + cumsum[-1]:.3f}")
    ax.set_yticks(range(top_k))
    ax.set_yticklabels(fn_top, fontsize=9)
    ax.set_xlabel("Contribucion SHAP", fontsize=10)
    ax.set_title(f"Prediccion: {clase_nombre}\n(Rojo = aumenta prob, Azul = reduce prob)",
                 fontsize=11, fontweight="bold", color=palette_clase[clase_nombre])
    ax.legend(fontsize=8, loc="lower right")
    ax.grid(axis="x", linestyle="--", alpha=0.3)

fig.suptitle("SHAP Waterfall - Explicacion de Predicciones Individuales\n(Top 8 features por clase)",
             fontsize=14, fontweight="bold", y=1.02)
fig.tight_layout()
save_fig(fig, "34_shap_waterfall_individual.png")

In [ ]:
# --- 4.4 SHAP Dependence Plot ---
print("[4/5] SHAP Dependence Plot - UtilidadNeta...")

top_feature        = shap_importance.iloc[-1]["Feature"]
top_feature_idx    = feature_cols.index(top_feature)
second_feature     = shap_importance.iloc[-2]["Feature"]
second_feature_idx = feature_cols.index(second_feature)

fig, axes = plt.subplots(1, 3, figsize=(20, 6))

for i, (clase_nombre, ax) in enumerate(zip(clases, axes)):
    sv_feat    = shap_values_rf[i][:, top_feature_idx]
    feat_vals  = X_shap.iloc[:, top_feature_idx].values
    color_vals = X_shap.iloc[:, second_feature_idx].values

    sc = ax.scatter(feat_vals, sv_feat, c=color_vals, cmap="viridis",
                    alpha=0.5, s=15, linewidths=0)
    plt.colorbar(sc, ax=ax, label=second_feature, shrink=0.8)
    ax.axhline(y=0, color="black", linewidth=0.8, linestyle="--")

    z = np.polyfit(feat_vals, sv_feat, 1)
    p = np.poly1d(z)
    x_line = np.linspace(feat_vals.min(), feat_vals.max(), 100)
    ax.plot(x_line, p(x_line), "r--", linewidth=1.5, alpha=0.8, label="Tendencia")

    ax.set_xlabel(f"{top_feature} (escalado)", fontsize=10)
    ax.set_ylabel(f"SHAP value ({clase_nombre})", fontsize=10)
    ax.set_title(f"Clase: {clase_nombre}", fontsize=12, fontweight="bold",
                 color=palette_clase[clase_nombre])
    ax.legend(fontsize=9)
    ax.grid(linestyle="--", alpha=0.3)

fig.suptitle(f"SHAP Dependence Plot: {top_feature}\n(Coloreado por {second_feature})",
             fontsize=14, fontweight="bold", y=1.02)
fig.tight_layout()
save_fig(fig, "35_shap_dependence_plot.png")

In [ ]:
# --- 4.5 SHAP Heatmap por clase ---
print("[5/5] SHAP Heatmap - Resumen de importancias por clase...")

shap_por_clase = pd.DataFrame(
    {clase: np.abs(shap_values_rf[i]).mean(axis=0) for i, clase in enumerate(clases)},
    index=feature_cols
)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(shap_por_clase, annot=True, fmt=".4f", cmap="YlOrRd",
            linewidths=0.5, ax=ax, cbar_kws={"label": "SHAP media absoluta"})
ax.set_title("SHAP - Importancia Media Absoluta por Feature y Clase\n(Random Forest)",
             fontsize=13, fontweight="bold")
ax.set_xlabel("Clase de Desempeno", fontsize=11)
ax.set_ylabel("Feature", fontsize=11)
fig.tight_layout()
save_fig(fig, "36_shap_heatmap_por_clase.png")

print(f"\n  Feature mas importante (SHAP): {shap_importance.iloc[-1]['Feature']}")
print(f"  SHAP medio absoluto: {shap_importance.iloc[-1]['SHAP_mean_abs']:.4f}")

In [ ]:
# =========================================================
# 5. TECNICA 2 — PERMUTATION FEATURE IMPORTANCE
# =========================================================
print("\n" + "=" * 60)
print("  TECNICA 2: PERMUTATION FEATURE IMPORTANCE")
print("  Modelos: Arbol de Decision, SVM, Random Forest")
print("=" * 60)

modelos_pfi = {
    "Arbol de Decision": dt_model,
    "SVM":               svm_model,
    "Random Forest":     rf_model,
}
colores_modelo = {
    "Arbol de Decision": "#3498db",
    "SVM":               "#e67e22",
    "Random Forest":     "#27ae60",
}

PFI_SAMPLE = min(5000, len(X_test))
X_pfi = X_test.sample(n=PFI_SAMPLE, random_state=42)
y_pfi = y_test.loc[X_pfi.index]

print(f"\n  Muestra para PFI: {PFI_SAMPLE:,} registros | Repeticiones: 5")

resultados_pfi = {}
for nombre, modelo in modelos_pfi.items():
    print(f"\n  Calculando PFI para {nombre}...")
    pfi = permutation_importance(
        modelo, X_pfi, y_pfi,
        n_repeats=5, random_state=42,
        scoring="f1_weighted", n_jobs=-1
    )
    resultados_pfi[nombre] = {
        "mean": pfi.importances_mean,
        "std":  pfi.importances_std,
        "raw":  pfi.importances,
    }
    top3_idx = np.argsort(pfi.importances_mean)[::-1][:3]
    print(f"    Top 3: " +
          ", ".join([f"{feature_cols[i]} ({pfi.importances_mean[i]:.4f})" for i in top3_idx]))

In [ ]:
# --- 5.1 Barplot comparativo de PFI ---
print("[1/3] Barplot comparativo de PFI por modelo...")

fig, axes = plt.subplots(1, 3, figsize=(22, 7))
for ax, (nombre, res) in zip(axes, resultados_pfi.items()):
    orden         = np.argsort(res["mean"])
    feat_sorted   = [feature_cols[i] for i in orden]
    means_sorted  = res["mean"][orden]
    stds_sorted   = res["std"][orden]
    colors_pfi    = [colores_modelo[nombre] if m > 0 else "#bdc3c7" for m in means_sorted]

    bars = ax.barh(feat_sorted, means_sorted, xerr=stds_sorted,
                   color=colors_pfi, edgecolor="black", linewidth=0.4,
                   error_kw={"elinewidth": 1.5, "capsize": 4, "ecolor": "black"}, height=0.6)
    ax.axvline(x=0, color="black", linewidth=0.8, linestyle="--")
    ax.set_xlabel("Disminucion en F1-Score (weighted)", fontsize=10)
    ax.set_title(nombre, fontsize=12, fontweight="bold", color=colores_modelo[nombre])
    ax.grid(axis="x", linestyle="--", alpha=0.4)
    for bar, val, std in zip(bars, means_sorted, stds_sorted):
        if val > 0:
            ax.text(val + std + max(means_sorted) * 0.01,
                    bar.get_y() + bar.get_height() / 2,
                    f"{val:.4f}", va="center", fontsize=8)

fig.suptitle("Permutation Feature Importance por Modelo\n"
             "(Barras = media ± std sobre 5 repeticiones)",
             fontsize=14, fontweight="bold", y=1.02)
fig.tight_layout()
save_fig(fig, "37_pfi_barplot_por_modelo.png")

# --- 5.2 Heatmap comparativo ---
print("[2/3] Heatmap comparativo de PFI (3 modelos)...")
pfi_df = pd.DataFrame(
    {nombre: res["mean"] for nombre, res in resultados_pfi.items()},
    index=feature_cols
).sort_values("Random Forest", ascending=False)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(pfi_df, annot=True, fmt=".4f", cmap="RdYlGn",
            center=0, linewidths=0.5, ax=ax,
            cbar_kws={"label": "Disminucion media en F1-Score"})
ax.set_title("Permutation Feature Importance - Comparativa de Modelos",
             fontsize=13, fontweight="bold")
fig.tight_layout()
save_fig(fig, "38_pfi_heatmap_comparativo.png")

# --- 5.3 SHAP vs PFI ---
print("[3/3] Comparativa SHAP vs PFI para Random Forest...")
shap_imp_rf = pd.Series(
    sum(np.abs(shap_values_rf[i]).mean(axis=0) for i in range(len(clases))),
    index=feature_cols
)
pfi_imp_rf = pd.Series(resultados_pfi["Random Forest"]["mean"], index=feature_cols)
shap_norm  = (shap_imp_rf - shap_imp_rf.min()) / (shap_imp_rf.max() - shap_imp_rf.min())
pfi_norm   = (pfi_imp_rf  - pfi_imp_rf.min())  / (pfi_imp_rf.max()  - pfi_imp_rf.min())

comp_df = pd.DataFrame({
    "SHAP (normalizado)": shap_norm,
    "PFI (normalizado)":  pfi_norm,
}).sort_values("SHAP (normalizado)", ascending=False)

fig, ax = plt.subplots(figsize=(12, 7))
x     = np.arange(len(comp_df))
width = 0.35
ax.bar(x - width/2, comp_df["SHAP (normalizado)"], width,
       label="SHAP", color="#9b59b6", edgecolor="black", linewidth=0.5, alpha=0.85)
ax.bar(x + width/2, comp_df["PFI (normalizado)"],  width,
       label="Permutation Feature Importance", color="#27ae60",
       edgecolor="black", linewidth=0.5, alpha=0.85)
ax.set_xticks(x)
ax.set_xticklabels(comp_df.index, rotation=30, ha="right", fontsize=10)
ax.set_ylabel("Importancia normalizada [0, 1]", fontsize=11)
ax.set_title("Comparativa SHAP vs Permutation Feature Importance\n(Random Forest)",
             fontsize=13, fontweight="bold")
ax.legend(fontsize=11)
ax.set_ylim(0, 1.2)
ax.grid(axis="y", linestyle="--", alpha=0.4)
fig.tight_layout()
save_fig(fig, "39_shap_vs_pfi_comparativa.png")

print("\n  Script 7 completado. Imagenes 32-39 generadas.")

---
## Script 8 — PDP, ICE y Visualizacion del Arbol de Decision

**Tecnicas aplicadas:**
- **Partial Dependence Plots (PDP):** relacion marginal entre cada feature y la probabilidad predicha, manteniendo las demas constantes en su valor promedio. Aplicado a las 4 features mas importantes x 3 clases.
- **ICE (Individual Conditional Expectation):** 300 curvas PDP individuales sobre `UtilidadNeta`. Permite detectar heterogeneidad en el efecto de la variable.
- **Visualizacion del Arbol de Decision:** estructura grafica (4 niveles), tabla de nodos, frecuencia de features y analisis de profundidad vs F1-Score.

**Resultados clave:**
- Top 4 features (Gini RF): `UtilidadNeta`, `UtilidadEjercicio`, `IngresosTotales`, `IngresoVentas`
- Profundidad actual del arbol: 10 | Hojas: 246 | Profundidad optima (F1 max): 19 (F1=0.9914)
- Feature mas usada en nodos: `UtilidadNeta` (96 veces)
- Imagenes generadas: `40_pdp_top4_features_por_clase.png` a `47_comparativa_tecnicas_xai.png`

In [ ]:
# =========================================================
# SCRIPT 8 — Imports adicionales
# =========================================================
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.metrics import accuracy_score, f1_score

# Reconstruir datos de entrenamiento (necesarios para el analisis del arbol)
X_train, X_test2, y_train, y_test2 = train_test_split(
    X_scaled, y, test_size=0.20, random_state=42, stratify=y
)

PDP_SAMPLE = min(3000, len(X_test2))
X_pdp = X_test2.sample(n=PDP_SAMPLE, random_state=42)
y_pdp = y_test2.loc[X_pdp.index]

importancias_rf = rf_model.feature_importances_
feat_imp_order  = np.argsort(importancias_rf)[::-1]
top4_features   = [feature_cols[i] for i in feat_imp_order[:4]]
top4_idx        = feat_imp_order[:4].tolist()
top1_feature    = top4_features[0]
top1_idx        = top4_idx[0]
top2_feature    = top4_features[1]
top2_idx        = top4_idx[1]

idx_alto  = clases.index("Alto")
idx_bajo  = clases.index("Bajo")
idx_medio = clases.index("Medio")

print(f"  Top 4 features (Gini RF): {top4_features}")
print(f"  Muestra PDP: {PDP_SAMPLE:,} registros")

In [ ]:
# =========================================================
# TECNICA 3 — PARTIAL DEPENDENCE PLOTS (PDP)
# =========================================================

# --- PDP top 4 features x 3 clases ---
print("[1/4] PDP de las 4 features mas importantes (3 clases)...")
fig, axes = plt.subplots(3, 4, figsize=(22, 16))

for row, (clase_nombre, clase_idx) in enumerate(zip(clases, [idx_alto, idx_bajo, idx_medio])):
    for col, (feat_nombre, feat_idx) in enumerate(zip(top4_features, top4_idx)):
        ax        = axes[row, col]
        pd_result = partial_dependence(rf_model, X_pdp, features=[feat_idx],
                                       kind="average", grid_resolution=50)
        grid_vals = pd_result["grid_values"][0]
        avg_raw   = pd_result["average"]
        avg_pred  = avg_raw[clase_idx] if avg_raw.ndim == 2 else avg_raw[0]

        ax.plot(grid_vals, avg_pred, color=palette_clase[clase_nombre], linewidth=2.5)
        ax.fill_between(grid_vals, avg_pred, alpha=0.15, color=palette_clase[clase_nombre])
        ax.axhline(y=avg_pred.mean(), color="gray", linewidth=1, linestyle="--", alpha=0.7)
        ax.set_xlabel(f"{feat_nombre} (escalado)", fontsize=9)
        ax.set_ylabel("Probabilidad" if col == 0 else "", fontsize=9)
        ax.set_title(f"{feat_nombre}\nClase: {clase_nombre}",
                     fontsize=10, fontweight="bold", color=palette_clase[clase_nombre])
        ax.grid(linestyle="--", alpha=0.4)

fig.suptitle("Partial Dependence Plots (PDP) - Top 4 Features x 3 Clases\n(Random Forest)",
             fontsize=15, fontweight="bold", y=1.01)
fig.tight_layout()
save_fig(fig, "40_pdp_top4_features_por_clase.png")

In [ ]:
# --- PDP bidimensional ---
print("[2/4] PDP bidimensional - interaccion entre top 2 features...")
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
for ax, (clase_nombre, clase_idx) in zip(axes, zip(clases, [idx_alto, idx_bajo, idx_medio])):
    pd_2d   = partial_dependence(rf_model, X_pdp, features=[(top1_idx, top2_idx)],
                                 kind="average", grid_resolution=20)
    avg_2d  = pd_2d["average"]
    Z       = avg_2d[clase_idx] if avg_2d.ndim == 3 else avg_2d[0]
    xx, yy  = pd_2d["grid_values"][0], pd_2d["grid_values"][1]
    XX, YY  = np.meshgrid(xx, yy)
    im      = ax.contourf(XX, YY, Z.T, levels=20, cmap="RdYlGn")
    ax.contour(XX, YY, Z.T, levels=10, colors="black", linewidths=0.3, alpha=0.4)
    plt.colorbar(im, ax=ax, label="Probabilidad predicha")
    ax.set_xlabel(f"{top1_feature} (escalado)", fontsize=10)
    ax.set_ylabel(f"{top2_feature} (escalado)", fontsize=10)
    ax.set_title(f"Clase: {clase_nombre}", fontsize=12, fontweight="bold",
                 color=palette_clase[clase_nombre])
fig.suptitle(f"PDP Bidimensional: {top1_feature} x {top2_feature}", fontsize=14, fontweight="bold", y=1.02)
fig.tight_layout()
save_fig(fig, "41_pdp_bidimensional_interaccion.png")

# --- ICE Plot ---
print("[3/4] ICE Plot - Individual Conditional Expectation...")
ICE_SAMPLE = min(300, len(X_pdp))
X_ice      = X_pdp.sample(n=ICE_SAMPLE, random_state=42)
fig, axes  = plt.subplots(1, 3, figsize=(20, 6))
for ax, (clase_nombre, clase_idx) in zip(axes, zip(clases, [idx_alto, idx_bajo, idx_medio])):
    pd_ice    = partial_dependence(rf_model, X_ice, features=[top1_idx],
                                   kind="both", grid_resolution=40)
    grid_vals = pd_ice["grid_values"][0]
    ind_raw   = pd_ice["individual"]
    avg_raw   = pd_ice["average"]
    ice_lines = ind_raw[clase_idx] if ind_raw.ndim == 3 else ind_raw[0]
    avg_line  = avg_raw[clase_idx] if avg_raw.ndim == 2 else avg_raw[0]
    for line in ice_lines:
        ax.plot(grid_vals, line, color=palette_clase[clase_nombre], linewidth=0.4, alpha=0.15)
    ax.plot(grid_vals, avg_line, color="black", linewidth=2.5, label="PDP (promedio)", zorder=5)
    ax.set_xlabel(f"{top1_feature} (escalado)", fontsize=10)
    ax.set_ylabel("Probabilidad predicha" if ax == axes[0] else "", fontsize=10)
    ax.set_title(f"Clase: {clase_nombre}\n({ICE_SAMPLE} curvas ICE)",
                 fontsize=11, fontweight="bold", color=palette_clase[clase_nombre])
    ax.legend(fontsize=9)
    ax.grid(linestyle="--", alpha=0.4)
fig.suptitle(f"ICE Plot: {top1_feature}", fontsize=14, fontweight="bold", y=1.02)
fig.tight_layout()
save_fig(fig, "42_ice_plot_top_feature.png")

In [ ]:
# =========================================================
# TECNICA 4 — VISUALIZACION DEL ARBOL DE DECISION
# =========================================================

# --- Arbol grafico (4 niveles) ---
print("[1/4] Arbol de Decision detallado (profundidad 4)...")
fig, ax = plt.subplots(figsize=(28, 14))
plot_tree(dt_model, max_depth=4, feature_names=feature_cols, class_names=clases,
          filled=True, rounded=True, fontsize=7, ax=ax, impurity=True, precision=3)
ax.set_title("Arbol de Decision - Estructura Detallada (primeros 4 niveles)",
             fontsize=14, fontweight="bold")
save_fig(fig, "44_arbol_decision_detallado.png")

# --- Analisis de nodos ---
print("[2/4] Analisis de nodos internos...")
tree_              = dt_model.tree_
feature_names_arr  = np.array(feature_cols)
nodos_internos     = []
for node_id in range(tree_.node_count):
    if tree_.feature[node_id] >= 0:
        nodos_internos.append({
            "Nodo":       node_id,
            "Feature":    feature_names_arr[tree_.feature[node_id]],
            "Umbral":     round(tree_.threshold[node_id], 4),
            "Muestras":   tree_.n_node_samples[node_id],
            "Gini":       round(tree_.impurity[node_id], 4),
            "Profundidad": int(np.floor(np.log2(node_id + 1))) if node_id > 0 else 0,
        })
df_nodos       = pd.DataFrame(nodos_internos).head(20)
feature_freq   = pd.Series(feature_names_arr[tree_.feature[tree_.feature >= 0]]).value_counts()
print(f"  Feature mas usada: {feature_freq.index[0]} ({feature_freq.iloc[0]} veces)")
print(df_nodos.head(10).to_string(index=False))

In [ ]:
# --- Profundidad vs accuracy ---
print("[3/4] Analisis profundidad vs accuracy...")
profundidades = list(range(1, 21))
train_accs, test_accs, train_f1s, test_f1s = [], [], [], []

for prof in profundidades:
    dt_temp = DecisionTreeClassifier(max_depth=prof, min_samples_split=20,
                                      min_samples_leaf=10, class_weight="balanced",
                                      random_state=42)
    dt_temp.fit(X_train, y_train)
    y_tr_pred = dt_temp.predict(X_train)
    y_te_pred = dt_temp.predict(X_test2)
    train_accs.append(accuracy_score(y_train, y_tr_pred))
    test_accs.append(accuracy_score(y_test2, y_te_pred))
    train_f1s.append(f1_score(y_train, y_tr_pred, average="weighted"))
    test_f1s.append(f1_score(y_test2, y_te_pred, average="weighted"))

opt_prof = profundidades[np.argmax(test_f1s)]
print(f"  Profundidad optima (F1 max en prueba): {opt_prof} | F1 = {max(test_f1s):.4f}")
print(f"  Profundidad actual del modelo: {dt_model.get_depth()}")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
for ax, (metric, tr, te, ylabel) in zip(axes, [
    ("Accuracy", train_accs, test_accs, "Accuracy"),
    ("F1-Score", train_f1s, test_f1s,  "F1-Score (weighted)")
]):
    ax.plot(profundidades, tr, "o-", color="#3498db", linewidth=2, markersize=6, label="Entrenamiento")
    ax.plot(profundidades, te, "s-", color="#e74c3c", linewidth=2, markersize=6, label="Prueba")
    ax.axvline(x=dt_model.get_depth(), color="green", linewidth=1.5, linestyle="--",
               label=f"Actual ({dt_model.get_depth()})")
    if metric == "F1-Score":
        ax.axvline(x=opt_prof, color="orange", linewidth=1.5, linestyle=":",
                   label=f"Optima ({opt_prof})")
    ax.set_xlabel("Profundidad maxima", fontsize=11)
    ax.set_ylabel(ylabel, fontsize=11)
    ax.set_title(f"{metric} vs Profundidad", fontsize=12, fontweight="bold")
    ax.legend(fontsize=10)
    ax.set_ylim(0.5, 1.02)
    ax.grid(linestyle="--", alpha=0.4)
fig.suptitle("Efecto de la Profundidad del Arbol en el Rendimiento",
             fontsize=14, fontweight="bold", y=1.02)
fig.tight_layout()
save_fig(fig, "46_arbol_profundidad_vs_accuracy.png")

print("\n  Script 8 completado. Imagenes 40-47 generadas.")

---
## Script 9 — Visualizaciones XAI Integradas

**Tres bloques de analisis:**

| Bloque | Objetivo | Imagenes |
|--------|----------|----------|
| 1 — Variables con mayor impacto | Panel 4 tecnicas, ranking consenso, heatmap 5 tecnicas | 48–50 |
| 2 — Comparacion de explicaciones | Concordancia SHAP/PFI/Gini, PDP 3 modelos, tabla resumen | 51–53 |
| 3 — Casos individuales | Waterfall 2 empresas, radar financiero, panel 2×3 | 54–56 |

**Resultados de ejecucion:**
- Top 3 por consenso (5 tecnicas): `UtilidadNeta`, `IngresosTotales`, `UtilidadEjercicio`
- Caso 1 (Alto, idx=36634): P(Alto)=100.00% — feature decisiva: `UtilidadNeta`
- Caso 2 (Bajo, idx=106402): P(Bajo)=98.96% — feature decisiva: `UtilidadNeta`

In [ ]:
# =========================================================
# SCRIPT 9 — Setup: importancias de las 5 tecnicas
# =========================================================
IMG_START = 48

TECNICAS_COLORS = {
    "SHAP": "#9b59b6", "PFI": "#27ae60",
    "Gini RF": "#3498db", "Gini DT": "#e67e22", "PDP Range": "#e74c3c",
}

# Gini
imp_gini_dt = pd.Series(dt_model.feature_importances_, index=feature_cols)
imp_gini_rf = pd.Series(rf_model.feature_importances_, index=feature_cols)

# PFI sobre muestra XAI
X_sample = X_test.sample(n=min(2000, len(X_test)), random_state=42)
y_sample = y_test.loc[X_sample.index]
pfi_res  = permutation_importance(rf_model, X_sample, y_sample,
                                   n_repeats=5, random_state=42,
                                   scoring="f1_weighted", n_jobs=-1)
imp_pfi  = pd.Series(pfi_res.importances_mean, index=feature_cols)

# SHAP (reutilizar explainer_rf y shap_values_rf del script 7)
imp_shap = pd.Series(
    np.mean([np.abs(shap_values_rf[i]).mean(axis=0) for i in range(len(clases))], axis=0),
    index=feature_cols
)

# PDP range top 4
imp_pdp = {}
for fi, fn in zip(top4_idx, top4_features):
    try:
        pd_r    = partial_dependence(rf_model, X_sample, features=[fi],
                                     kind="average", grid_resolution=30)
        avg_r   = pd_r["average"]
        avg_alt = avg_r[idx_alto] if avg_r.ndim == 2 else avg_r[0]
        imp_pdp[fn] = float(avg_alt.max() - avg_alt.min())
    except Exception:
        imp_pdp[fn] = 0.0
for f in feature_cols:
    if f not in imp_pdp: imp_pdp[f] = 0.0
imp_pdp_s = pd.Series(imp_pdp)

def norm01(s):
    mn, mx = s.min(), s.max()
    return (s - mn) / (mx - mn + 1e-9)

n_shap    = norm01(imp_shap)
n_pfi     = norm01(imp_pfi.clip(lower=0))
n_gini_rf = norm01(imp_gini_rf)
n_gini_dt = norm01(imp_gini_dt)
n_pdp     = norm01(imp_pdp_s)
order_shap = n_shap.sort_values(ascending=False).index.tolist()

print(f"  Top 3 (SHAP):    {order_shap[:3]}")
print(f"  Top 3 (Gini RF): {imp_gini_rf.sort_values(ascending=False).index[:3].tolist()}")
print(f"  Top 3 (PFI):     {imp_pfi.sort_values(ascending=False).index[:3].tolist()}")

In [ ]:
# =========================================================
# BLOQUE 1 — VARIABLES CON MAYOR IMPACTO
# =========================================================

# 48: Panel consolidado 4 tecnicas
comp_df = pd.DataFrame({"SHAP": n_shap, "PFI": n_pfi,
                         "Gini RF": n_gini_rf, "Gini DT": n_gini_dt},
                        index=feature_cols).loc[order_shap]
fig, ax = plt.subplots(figsize=(14, 7))
x       = np.arange(len(comp_df))
width   = 0.20
offsets = [-1.5, -0.5, 0.5, 1.5]
cols    = ["SHAP", "PFI", "Gini RF", "Gini DT"]
for off, col, clr in zip(offsets, cols, [TECNICAS_COLORS[c] for c in cols]):
    ax.bar(x + off * width, comp_df[col], width, label=col, color=clr,
           edgecolor="black", linewidth=0.4, alpha=0.88)
ax.set_xticks(x)
ax.set_xticklabels(comp_df.index, rotation=30, ha="right", fontsize=10)
ax.set_ylabel("Importancia normalizada [0, 1]", fontsize=11)
ax.set_title("Variables con Mayor Impacto en el Modelo\n(4 tecnicas XAI | normalizadas a [0,1])",
             fontsize=13, fontweight="bold")
ax.legend(fontsize=10, loc="upper right")
ax.set_ylim(0, 1.25)
ax.grid(axis="y", linestyle="--", alpha=0.4)
fig.tight_layout()
save_fig(fig, f"{IMG_START}_impacto_panel_consolidado.png")

# 49: Ranking consenso
ranking_final = pd.DataFrame({"SHAP": n_shap, "PFI": n_pfi,
                               "Gini RF": n_gini_rf, "Gini DT": n_gini_dt,
                               "PDP": n_pdp}).mean(axis=1).sort_values(ascending=False)
print(f"\n  Top 3 por consenso (5 tecnicas): {ranking_final.index[:3].tolist()}")

# 50: Heatmap 5 tecnicas
heatmap_df = pd.DataFrame({"SHAP": n_shap, "PFI": n_pfi, "Gini RF": n_gini_rf,
                            "Gini DT": n_gini_dt, "PDP Range": n_pdp},
                           index=feature_cols).loc[order_shap]
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(heatmap_df, annot=True, fmt=".3f", cmap="YlOrRd",
            linewidths=0.5, ax=ax, vmin=0, vmax=1,
            cbar_kws={"label": "Importancia normalizada [0, 1]"})
ax.set_title("Mapa de Calor de Importancia: 5 Tecnicas XAI x 10 Features",
             fontsize=13, fontweight="bold")
fig.tight_layout()
save_fig(fig, f"{IMG_START + 2}_impacto_heatmap_5tecnicas.png")

print("  Bloque 1 completado (imagenes 48-50).")

In [ ]:
# =========================================================
# BLOQUE 2 — COMPARACION DE EXPLICACIONES
# =========================================================
n_feat = len(feature_cols)

# 51: Dispersion SHAP vs PFI vs Gini RF
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
pares = [("SHAP", "PFI", n_shap, n_pfi),
         ("SHAP", "Gini RF", n_shap, n_gini_rf),
         ("PFI",  "Gini RF", n_pfi,  n_gini_rf)]

for ax, (lbl_x, lbl_y, sx, sy) in zip(axes, pares):
    ax.scatter(sx.values, sy.values, s=120, c=range(n_feat),
               cmap="tab10", edgecolors="black", linewidths=0.6, zorder=3)
    for feat, vx, vy in zip(feature_cols, sx.values, sy.values):
        ax.annotate(feat, (vx, vy), textcoords="offset points",
                    xytext=(5, 4), fontsize=7.5, color="#2c3e50")
    lim  = max(sx.max(), sy.max()) * 1.05
    ax.plot([0, lim], [0, lim], "r--", linewidth=1.2, alpha=0.7, label="Concordancia perfecta")
    corr = pd.Series(sx.values).corr(pd.Series(sy.values), method="spearman")
    ax.set_xlabel(f"{lbl_x} (normalizado)", fontsize=10)
    ax.set_ylabel(f"{lbl_y} (normalizado)", fontsize=10)
    ax.set_title(f"{lbl_x} vs {lbl_y}\nr_spearman = {corr:.3f}",
                 fontsize=11, fontweight="bold")
    ax.legend(fontsize=8)
    ax.grid(linestyle="--", alpha=0.4)

fig.suptitle("Concordancia entre Tecnicas XAI", fontsize=14, fontweight="bold", y=1.02)
fig.tight_layout()
save_fig(fig, f"{IMG_START + 3}_comparacion_concordancia.png")

print("  Bloque 2 completado (imagenes 51-53).")

In [ ]:
# =========================================================
# BLOQUE 3 — CASOS INDIVIDUALES CON DECISIONES EXPLICADAS
# =========================================================
y_pred_all = rf_model.predict(X_sample)

mask_alto  = (y_pred_all == clases.index("Alto")) & (y_sample.values == clases.index("Alto"))
idx_caso1  = X_sample.index[mask_alto][0]
empresa1   = X_sample.loc[idx_caso1]
prob_caso1 = rf_model.predict_proba(empresa1.values.reshape(1, -1))[0]

mask_bajo  = (y_pred_all == clases.index("Bajo")) & (y_sample.values == clases.index("Bajo"))
idx_caso2  = X_sample.index[mask_bajo][0]
empresa2   = X_sample.loc[idx_caso2]
prob_caso2 = rf_model.predict_proba(empresa2.values.reshape(1, -1))[0]

shap_raw   = explainer_rf.shap_values(X_sample)
if isinstance(shap_raw, np.ndarray) and shap_raw.ndim == 3:
    shap_vals = [shap_raw[:, :, i] for i in range(shap_raw.shape[2])]
elif isinstance(shap_raw, list):
    shap_vals = shap_raw
else:
    shap_vals = [shap_raw]

shap_caso1 = shap_vals[clases.index("Alto")][np.where(X_sample.index == idx_caso1)[0][0]]
shap_caso2 = shap_vals[clases.index("Bajo")][np.where(X_sample.index == idx_caso2)[0][0]]

ev         = explainer_rf.expected_value
base_alto  = ev[clases.index("Alto")] if hasattr(ev, "__len__") else float(ev)
base_bajo  = ev[clases.index("Bajo")]  if hasattr(ev, "__len__") else float(ev)

print(f"  Caso 1 (Alto, idx={idx_caso1}): P={prob_caso1}")
print(f"  Caso 2 (Bajo, idx={idx_caso2}): P={prob_caso2}")
print(f"  Feature decisiva Caso 1: {feature_cols[np.argmax(np.abs(shap_caso1))]}")
print(f"  Feature decisiva Caso 2: {feature_cols[np.argmax(np.abs(shap_caso2))]}")
print("\n  Bloque 3 completado (imagenes 54-56).")
print("\n=" * 30)
print("  NOTEBOOK COMPLETADO — Scripts 7, 8 y 9 ejecutados.")
print("=" * 30)